# Data Analysis for "Heroes of Pymoli"

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Importing file and pre-viewing the data
purchasedf=pd.read_csv("Resources/purchase_data.csv")
purchasedf.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#basic statistics 
purchasedf.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,92.114103,3.050987
std,225.310896,6.659444,52.775943,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,48.000000,1.980000
50%,389.500000,22.000000,93.000000,3.150000
75%,584.250000,25.000000,139.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


In [4]:
#variable types in the df and checking for missing values and droping duplicates for Gender analysis
purchasedf.info()
purchasedf.isnull().sum() 
select_gender=purchasedf[['SN','Gender','Age']].drop_duplicates()
select_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
Purchase ID    780 non-null int64
SN             780 non-null object
Age            780 non-null int64
Gender         780 non-null object
Item ID        780 non-null int64
Item Name      780 non-null object
Price          780 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 778
Data columns (total 3 columns):
SN        576 non-null object
Gender    576 non-null object
Age       576 non-null int64
dtypes: int64(1), object(2)
memory usage: 18.0+ KB


# Player Count

In [5]:
#Total of players, another alternative is #len(purchasedf["SN"].value_counts())
total_players=purchasedf["SN"].nunique()
#print(f'Total Players:{total_players}')
totalp={"Total Players":[total_players]}
total=pd.DataFrame(totalp)
total

,Total Players
0,576


# Purchasing Analysis (Total)

In [6]:
Purchasing_analysis_dic = {'Number of Unique Items': [len(purchasedf['Item ID'].value_counts())],
                           'Average Price': [purchasedf['Price'].mean()],
                           'Number of Purchases': [len(purchasedf['Purchase ID'].value_counts())],
                           'Total Revenue': [purchasedf['Price'].sum()]}


Purchasing_analysis = pd.DataFrame (Purchasing_analysis_dic)

In [7]:
Purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.050987,780,2379.77


# Gender demographics

In [8]:
#select_gender.head()
Demographics=select_gender.groupby("Gender").count()
Demographics.columns
Demographics.rename(columns={"SN":"Total Count","Age":"Percentage of Players"},inplace=True)
for i in range( len (Demographics)):
    Demographics.iloc[i]["Percentage of Players"]=Demographics.iloc[i]["Total Count"]/np.sum(Demographics["Total Count"])*100

    

In [9]:
Demographics

,Total Count,Percentage of Players
Gender,,
Female,81,14
Male,484,84
Other / Non-Disclosed,11,1


## Purchasing Analysis (Gender)


In [10]:
p_count=pd.DataFrame(purchasedf.groupby("Gender")["Item ID"].count())
ave_price=pd.DataFrame(purchasedf.groupby("Gender")["Price"].mean()).rename(columns={"Price":"Average Purchase Price"})
total_v=pd.DataFrame(purchasedf.groupby("Gender")["Price"].sum())
ave_t_price=pd.DataFrame(total_v["Price"]/Demographics["Total Count"])


In [11]:
#merging
gender_analysis=pd.concat([p_count,ave_price,total_v,ave_t_price],axis=1)
gender_analysis.columns

gender_analysis.rename(columns={"Item ID":"Purchase Count",
                                "Price":"Total Purchase Value",
                                0:"Avg Total Purchase per Person"},inplace=True)
gender_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


# Age Demographics

In [12]:
bins=[0,9,14,19,24,29,34,39,50]
labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
select_gender["Age binned"]=pd.cut(select_gender["Age"],bins,labels=labels).sort_index()

byage=pd.DataFrame(select_gender["Age binned"].value_counts()).rename(columns={"Age binned":"Total Count"})
byage.sort_index(inplace=True)

byage['percent of players']=byage["Total Count"].apply(lambda x: x / total['Total Players'] * 100)


In [13]:
byage

,Total Count,percent of players
<10,17,2.951389
10-14,22,3.819444
15-19,107,18.576389
20-24,258,44.791667
25-29,77,13.368056
30-34,52,9.027778
35-39,31,5.381944
40+,12,2.083333


# Purchasing Analysis (Age)

In [14]:
bins=[0,9,14,19,24,29,34,39,50]
labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
purchasedf["Age binned"]=pd.cut(purchasedf["Age"],bins,labels=labels)
p_count_age=pd.DataFrame(purchasedf.groupby("Age binned")["Item ID"].count())
ave_price_age=pd.DataFrame(purchasedf.groupby("Age binned")["Price"].mean()).rename(columns={"Price":"Average Purchase Price"})
total_v_age=pd.DataFrame(purchasedf.groupby("Age binned")["Price"].sum())
ave_t_price_age=pd.DataFrame(total_v_age["Price"]/byage["Total Count"])



In [15]:
age_analysis=pd.concat([p_count_age,ave_price_age,total_v_age,ave_t_price_age],axis=1)
age_analysis.columns

age_analysis.rename(columns={"Item ID":"Purchase Count",
                                "Price":"Total Purchase Value",
                                0:"Avg Total Purchase per Person"},inplace=True)
age_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age binned,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


# Top 5 Spenders

In [16]:
purchase_top=pd.DataFrame(purchasedf.groupby("SN").count())
ave_price_top=pd.DataFrame(purchasedf.groupby("SN")["Price"].mean()).rename(columns={"Price":"Average Purchase Price"})
total_v_top=pd.DataFrame(purchasedf.groupby("SN")["Price"].sum()).rename(columns={"Price":"Total Purchase Value"})



In [19]:
top5=pd.concat([purchase_top,ave_price_top,total_v_top],axis=1)
top5.columns

top5.rename(columns={"Purchase ID":"Purchase Count"},inplace=True)
top5.sort_values("Total Purchase Value",ascending=False,inplace=True)
top5[["Purchase Count","Average Purchase Price","Total Purchase Value"]].head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


# Most Popular Items

In [37]:
purchase_pop_ite=pd.DataFrame(purchasedf.groupby("Item ID").count())

ave_price_pop_ite=pd.DataFrame(purchasedf.groupby("Item ID")["Price"].mean()).rename(columns={"Price":"Item Price"})
total_v_pop_ite=pd.DataFrame(purchasedf.groupby("Item ID")["Price"].sum()).rename(columns={"Price":"Total Purchase Value"})

popular5=pd.concat([purchase_pop_ite,ave_price_pop_ite,total_v_pop_ite,item_name],axis=1)
popular5.rename(columns={"Purchase ID":"Purchase Count"},inplace=True)
popular5.sort_values("Purchase Count",ascending=False,inplace=True)
popular5[["Purchase Count","Item Price","Total Purchase Value"]].head()

,Purchase Count,Item Price,Total Purchase Value
178,12.0,4.23,50.76
145,9.0,4.58,41.22
108,9.0,3.53,31.77
82,9.0,4.90,44.10
19,8.0,1.02,8.16


# Most Profitable Items

In [46]:
profitable=pd.concat([purchase_pop_ite,ave_price_pop_ite,total_v_pop_ite,item_name],axis=1)
profitable.rename(columns={"Purchase ID":"Purchase Count"},inplace=True)
profitable.sort_values("Total Purchase Value",ascending=False,inplace=True)
profitable[["Purchase Count","Item Price","Total Purchase Value"]].head()


,Purchase Count,Item Price,Total Purchase Value
178,12.0,4.23,50.76
82,9.0,4.90,44.10
145,9.0,4.58,41.22
92,8.0,4.88,39.04
103,8.0,4.35,34.80
